<a href="https://colab.research.google.com/github/cayang17/ds2003_GroupR/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Need to convert original XPT file to CSV

# Load the XPT file
df = pd.read_sas("/content/LLCP2017.XPT ")

# Save as CSV
df.to_csv("/content/BRFSS_2017.csv", index=False)

print("Conversion complete and file saved as BRFSS_2017.csv")
df.head()


Conversion complete! File saved as BRFSS_2017.csv


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01302017',b'01',b'30',b'2017',1100.0,b'2017000001',2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,b'01122017',b'01',b'12',b'2017',1100.0,b'2017000002',2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,b'01102017',b'01',b'10',b'2017',1100.0,b'2017000003',2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0
3,1.0,1.0,b'02082017',b'02',b'08',b'2017',1200.0,b'2017000004',2.017000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
4,1.0,1.0,b'01302017',b'01',b'30',b'2017',1100.0,b'2017000005',2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0


In [4]:
from google.colab import files
files.download("/content/BRFSS_2017.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import os
os.listdir('/content')

['.config', 'LLCP2017.XPT ', 'sample_data']

In [5]:
import pandas as pd

# Need to remove the columns that will not be used as original file is too large to upload to Github

# LOAD BRFSS
df = pd.read_csv("/content/BRFSS_2017.csv", low_memory=False)

print("Original shape:", df.shape)


# Select the columns used
keep_cols = [
    '_BMI5',
    '_AGE80',
    'SEX',
    '_IMPRACE',
    'INCOME2',
    'EXERANY2',
    'PA1MIN_',
    'SMOKE100',
    'SMOKDAY2',
    'ALCDAY5',
    'DRNK3GE5',
    'FRUIT2',
    'VEGETAB2',
    '_STATE',
    '_CNTYFIP'
]

# Filter only the columns that exist (some variables vary by year)
existing_cols = [c for c in keep_cols if c in df.columns]

df_small = df[existing_cols].copy()

print("Columns kept:", existing_cols)
print("Shape after keeping only needed columns:", df_small.shape)


# Clean data (drop rows missing key valuess
# Drop rows with missing BMI or county FIPS
critical_vars = ['_BMI5', '_STATE']
critical_vars = [c for c in critical_vars if c in df_small.columns]

df_small = df_small.dropna(subset=critical_vars)

# onvert special BRFSS missing codes (777=don’t know, 999=refused)
df_small.replace({777: None, 999: None, 888: None}, inplace=True)

print("Shape after cleaning:", df_small.shape)

# Create binary obesity outcome
# BRFSS stores BMI as BMI * 100. So 3000 = BMI of 30.
if '_BMI5' in df_small.columns:
    df_small['OBESE'] = (df_small['_BMI5'] >= 3000).astype(int)

# 5. Save cleaned dataset
output_path = "/content/CDCBRFSS_2017.csv"
df_small.to_csv(output_path, index=False)

print("Saved cleaned dataset to:", output_path)
df_small.head()


Original shape: (142820, 358)
Columns kept: ['_BMI5', '_AGE80', 'SEX', '_IMPRACE', 'INCOME2', 'EXERANY2', 'PA1MIN_', 'SMOKE100', 'SMOKDAY2', 'ALCDAY5', 'DRNK3GE5', 'FRUIT2', 'VEGETAB2', '_STATE']
Shape after keeping only needed columns: (142820, 14)
Shape after cleaning: (130842, 14)
Saved cleaned dataset to: /content/BRFSS_2017_CLEANED.csv


,_BMI5,_AGE80,SEX,_IMPRACE,INCOME2,EXERANY2,PA1MIN_,SMOKE100,SMOKDAY2,ALCDAY5,DRNK3GE5,FRUIT2,VEGETAB2,_STATE,OBESE
0,2696.0,74.0,2.0,1.0,6.0,1.0,360.0,2.0,NaN,None,NaN,102.0,101.0,1.0,0
1,2943.0,67.0,1.0,1.0,8.0,1.0,420.0,2.0,NaN,None,NaN,103.0,101.0,1.0,0
2,2504.0,70.0,1.0,1.0,99.0,2.0,NaN,2.0,NaN,101.0,88.0,202.0,201.0,1.0,0
3,2663.0,73.0,2.0,1.0,1.0,NaN,NaN,2.0,NaN,None,NaN,301.0,NaN,1.0,0
4,2330.0,69.0,2.0,1.0,2.0,2.0,NaN,1.0,3.0,None,NaN,101.0,303.0,1.0,0


In [6]:
from google.colab import files
files.download("/content/CDCBRFSS_2017.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>